# Evaluating BridgeStan Speed

This notebook is meant to evaluate the speed of BridgeStan relative to [issue #190](https://github.com/roualdes/bridgestan/issues/190). The goal of the issue is to offer users direct access Stan's methods via both numpy arrays and ctypes.  The solution to the issue under question uses one extra if statement to allow either numpy arrays or ctypes.  We anticipate the if statement to (potentially) cause a slow down from a numpy array user perspective.  From a ctypes user perspective the proposed solution is likely to be much faster than what exists in BridgeStan's main branch.  This notebook aims to quantify the potential slow down, for numpy array users, and the expected speed gain, for ctypes users.

### Clone and Install BridgeStan

In [1]:
# start fresh?
!rm -rf ./bridgestan

In [3]:
!git clone --recurse-submodules --shallow-submodules --depth=1 https://github.com/roualdes/bridgestan.git
!cd ./bridgestan && git fetch origin python/enable-ctypes-double-pointers:python/enable-ctypes-double-pointers && git checkout python/enable-ctypes-double-pointers

Switched to branch 'python/enable-ctypes-double-pointers'


In [4]:
!cd ./bridgestan && pip install ./python

Processing ./python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bridgestan: filename=bridgestan-2.2.2-py3-none-any.whl size=11744 sha256=d09633574848b1754e820cab17e213f65ed87799338b1d5311128ee983a09da1
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-f6okoqft/wheels/c0/af/43/4c10ee4e3df14332c19c8b6a53ff3b068ac02d4af336917a59
Successfully built bridgestan
  Attempting uninstall: bridgestan
    Found existing installation: bridgestan 2.2.2
    Uninstalling bridgestan-2.2.2:
      Successfully uninstalled bridgestan-2.2.2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Clone and Install ExperimentalHMC (for testing direct ctypes access)

In [5]:
!git clone git@github.com:roualdes/experimentalHMC.git
!cd ./experimentalHMC && git fetch origin start && git checkout start
!cd ./experimentalHMC && pip install .

fatal: destination path 'experimentalHMC' already exists and is not an empty directory.
From github.com:roualdes/experimentalHMC
 * branch            start      -> FETCH_HEAD
Already on 'start'
Your branch is up to date with 'origin/start'.
Processing /Users/ez/bridgestan-speed/experimentalHMC
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experimentalhmc: filename=experimentalhmc-0.0.1-cp311-cp311-macosx_12_0_x86_64.whl size=28055 sha256=632b3de592d7b7485ee617f830ebb3c187cec46b37b317fcc978d29558469d7c
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-xj2wmlix/wheels/db/d5/8f/6d7c68188c07281292ae0013dfd8cb0853d40aad60344fb904
Successfully built experimentalhmc
  Attempting uninstall: experimentalhmc
    Found existing installation: experimentalhmc 0.0.1
    Uninstalling experimentalhmc-0.0.1:
      Successfully uninstalled exper

### Evaluate Numpy Access

In [5]:
import bridgestan as bs
import numpy as np

bs.set_bridgestan_path("./bridgestan")

In [36]:
model = "simple"

m = bs.StanModel(f"test_models/{model}/{model}.stan",
                  data = f"test_models/{model}/{model}.data.json")

dims = m.param_unc_num()
R = 1_000
out = np.zeros(dims)
q = np.random.normal(size = (R, dims))

In [8]:
%%timeit -n 500
for r in range(R):
    m.log_density_gradient(q[r], out = out)

13.3 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [8]:
%%timeit -n 500
for r in range(R):
    m.log_density_gradient_proposed(q[r], out = out)

15.2 ms ± 575 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


### Evaluate Ctypes Access

In [10]:
import experimentalhmc as ehmc
from numpy.ctypeslib import as_array

In [29]:
def bridgestan_log_density_gradient_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient(as_array(position, shape = (dims,)), 
                                      out = as_array(gradient, shape = (dims,)))
        return ld
    return bsm_c_wrapper

def bridgestan_log_density_gradient_proposed_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient_proposed(position, out = gradient)
        return ld
    return bsm_c_wrapper

In [37]:
ldg = bridgestan_log_density_gradient_wrapper(m)
ldg_proposed = bridgestan_log_density_gradient_proposed_wrapper(m)

In [38]:
stan = ehmc.Stan(dims, ldg, seed = 204)
omv_stan = ehmc.OnlineMeanVar(dims)
stan_proposed = ehmc.Stan(dims, ldg_proposed, seed = 204)
omv_proposed = ehmc.OnlineMeanVar(dims)

In [39]:
%%timeit -n 20
for m in range(1000):
    x = stan.sample()
    omv_stan.update(x)

1.51 s ± 30.3 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [40]:
%%timeit -n 20
for m in range(1000):
    x = stan_proposed.sample()
    omv_proposed.update(x)

908 ms ± 92 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [34]:
(omv_stan.location(), omv_proposed.location())

(array([-1.20302244, -0.73516029,  0.4193706 , -0.41457035,  0.12638996,
        -0.36464968, -0.17881818, -0.15287945,  0.01286508,  0.18029888,
        -0.11071917, -0.22435393,  0.12277747,  0.02861073, -0.13612297,
        -0.29240139,  0.27868273, -0.29944824,  0.30408132,  0.27067469,
         0.12296306, -0.06271858, -0.0931013 , -0.02592632, -0.02339059]),
 array([-1.20302244, -0.73516029,  0.4193706 , -0.41457035,  0.12638996,
        -0.36464968, -0.17881818, -0.15287945,  0.01286508,  0.18029888,
        -0.11071917, -0.22435393,  0.12277747,  0.02861073, -0.13612297,
        -0.29240139,  0.27868273, -0.29944824,  0.30408132,  0.27067469,
         0.12296306, -0.06271858, -0.0931013 , -0.02592632, -0.02339059]))

In [35]:
(omv_stan.scale(), omv_proposed.scale())

(array([0.09211061, 0.09012227, 0.10420422, 0.09500876, 0.10805726,
        0.09462202, 0.09231088, 0.08181635, 0.09065747, 0.10413163,
        0.09734989, 0.0789365 , 0.09418236, 0.08595233, 0.0942715 ,
        0.11861248, 0.08279754, 0.10327394, 0.12105746, 0.1109676 ,
        0.13717264, 0.14309232, 0.09045763, 0.12779424, 0.12448835]),
 array([0.09211061, 0.09012227, 0.10420422, 0.09500876, 0.10805726,
        0.09462202, 0.09231088, 0.08181635, 0.09065747, 0.10413163,
        0.09734989, 0.0789365 , 0.09418236, 0.08595233, 0.0942715 ,
        0.11861248, 0.08279754, 0.10327394, 0.12105746, 0.1109676 ,
        0.13717264, 0.14309232, 0.09045763, 0.12779424, 0.12448835]))